# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_comp/repos/reward_competition_extention'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

Explanation of each input and where it comes from.

In [14]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [15]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/rce_pilot_2_02_full_spectral.pkl")

In [16]:
# CHANNEL_MAP_AND_ALL_TRIALS_DF = pd.read_pickle("./proc/full_baseline_and_trial_lfp_traces.pkl")
# MERGED_TRIALS_AND_VIDEO = pd.read_pickle("./proc/trial_SLEAP_and_metadata.pkl")

In [17]:
theta_band = (4, 12)
gamma_band = (30, 50)

In [18]:
FONTSIZE=20

In [19]:
FULL_LFP_TRACES_PKL = "{}_03_spectral_bands.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [20]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [21]:
LFP_SPECTRAL_DF.columns

Index(['cohort', 'session_dir', 'tone_frames', 'box_1_port_entry_frames',
       'box_2_port_entry_frames', 'video_name', 'session_path', 'recording',
       'current_subject', 'subject', 'all_subjects', 'first_timestamp',
       'last_timestamp', 'video_timestamps', 'tone_timestamps',
       'box_1_port_entry_timestamps', 'box_2_port_entry_timestamps',
       'lfp_timestamps', 'mPFC_lfp_trace', 'MD_lfp_trace', 'LH_lfp_trace',
       'BLA_lfp_trace', 'vHPC_lfp_trace', 'mPFC_lfp_zscore', 'MD_lfp_zscore',
       'LH_lfp_zscore', 'BLA_lfp_zscore', 'vHPC_lfp_zscore', 'mPFC_lfp_RMS',
       'MD_lfp_RMS', 'LH_lfp_RMS', 'BLA_lfp_RMS', 'vHPC_lfp_RMS',
       'mPFC_lfp_mask', 'MD_lfp_mask', 'LH_lfp_mask', 'BLA_lfp_mask',
       'vHPC_lfp_mask', 'mPFC_lfp_trace_filtered', 'MD_lfp_trace_filtered',
       'LH_lfp_trace_filtered', 'BLA_lfp_trace_filtered',
       'vHPC_lfp_trace_filtered', 'mPFC_lfp_RMS_filtered',
       'MD_lfp_RMS_filtered', 'LH_lfp_RMS_filtered', 'BLA_lfp_RMS_filtered',
       '

## Filtering by theta and gamma band

- Filtering theta/gamma for power

In [22]:
power_columns = [col for col in LFP_SPECTRAL_DF.columns if "power" in col and "calculation" not in col and "time" not in col]

In [23]:
power_columns

['mPFC_trace_power_all_frequencies_all_windows',
 'MD_trace_power_all_frequencies_all_windows',
 'LH_trace_power_all_frequencies_all_windows',
 'BLA_trace_power_all_frequencies_all_windows',
 'vHPC_trace_power_all_frequencies_all_windows',
 'mPFC_RMS_power_all_frequencies_all_windows',
 'MD_RMS_power_all_frequencies_all_windows',
 'LH_RMS_power_all_frequencies_all_windows',
 'BLA_RMS_power_all_frequencies_all_windows',
 'vHPC_RMS_power_all_frequencies_all_windows',
 'mPFC_trace_filtered_power_all_frequencies_all_windows',
 'MD_trace_filtered_power_all_frequencies_all_windows',
 'LH_trace_filtered_power_all_frequencies_all_windows',
 'BLA_trace_filtered_power_all_frequencies_all_windows',
 'vHPC_trace_filtered_power_all_frequencies_all_windows',
 'mPFC_RMS_filtered_power_all_frequencies_all_windows',
 'MD_RMS_filtered_power_all_frequencies_all_windows',
 'LH_RMS_filtered_power_all_frequencies_all_windows',
 'BLA_RMS_filtered_power_all_frequencies_all_windows',
 'vHPC_RMS_filtered_power_

In [24]:
LFP_SPECTRAL_DF["power_calculation_frequencies"].apply(lambda x: (x >= theta_band[0]) & (x <= theta_band[1])).iloc[0]

array([False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

In [25]:
for col in power_columns:
    # brain_region_name = col.split("_")[0]
    brain_region_name = col.split("power")[0]
    brain_region_name = brain_region_name.strip("_")
    print(brain_region_name)
    theta_power_col = "{}_power_theta".format(brain_region_name)
    gamma_power_col = "{}_power_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_power_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["power_calculation_frequencies"] >= theta_band[0]) & (x["power_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_power_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["power_calculation_frequencies"] >= gamma_band[0]) & (x["power_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

mPFC_trace
MD_trace
LH_trace
BLA_trace
vHPC_trace
mPFC_RMS
MD_RMS
LH_RMS
BLA_RMS
vHPC_RMS
mPFC_trace_filtered
MD_trace_filtered
LH_trace_filtered
BLA_trace_filtered
vHPC_trace_filtered
mPFC_RMS_filtered
MD_RMS_filtered
LH_RMS_filtered
BLA_RMS_filtered
vHPC_RMS_filtered


In [26]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,mPFC_RMS_filtered_power_theta,mPFC_RMS_filtered_power_gamma,MD_RMS_filtered_power_theta,MD_RMS_filtered_power_gamma,LH_RMS_filtered_power_theta,LH_RMS_filtered_power_gamma,BLA_RMS_filtered_power_theta,BLA_RMS_filtered_power_gamma,vHPC_RMS_filtered_power_theta,vHPC_RMS_filtered_power_gamma
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.00652, 0.002836, 0.002962, 0.00776, 0.01208...","[0.000125, 0.00016, 0.0001565, 0.0001225, 0.00...","[0.01796, 0.01675, 0.01249, 0.01534, 0.013664,...","[0.0003269, 0.0002108, 0.0003424, 0.0004702, 0...","[0.0123, 0.012405, 0.01134, 0.010544, 0.009544...","[0.0004094, 0.0003133, 0.000283, 0.0003424, 0....","[0.01605, 0.01243, 0.01147, 0.01507, 0.014824,...","[0.000981, 0.001048, 0.001005, 0.0007415, 0.00...","[0.006912, 0.0188, 0.03604, 0.01837, 0.010605,...","[0.000249, 0.000289, 0.0002723, 0.0003161, 0.0..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.00652, 0.002836, 0.002962, 0.00776, 0.01208...","[0.000125, 0.00016, 0.0001565, 0.0001225, 0.00...","[0.01796, 0.01675, 0.01249, 0.01534, 0.013664,...","[0.0003269, 0.0002108, 0.0003424, 0.0004702, 0...","[0.0123, 0.012405, 0.01134, 0.010544, 0.009544...","[0.0004094, 0.0003133, 0.000283, 0.0003424, 0....","[0.01605, 0.01243, 0.01147, 0.01507, 0.014824,...","[0.000981, 0.001048, 0.001005, 0.0007415, 0.00...","[0.006912, 0.0188, 0.03604, 0.01837, 0.010605,...","[0.000249, 0.000289, 0.0002723, 0.0003161, 0.0..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.01492, 0.01654, 0.03436, nan, nan, 0.01553,...","[0.001747, 0.002792, 0.00246, nan, nan, 0.0031...","[0.01246, 0.01306, 0.0218, 0.0201, 0.01401, 0....","[0.000678, 0.0005302, 0.0004344, 0.000489, 0.0...","[0.01442, 0.01605, 0.02498, 0.02408, 0.0166, 0...","[0.0008626, 0.0006566, 0.000616, 0.000811, 0.0...","[0.013916, nan, nan, 0.02411, nan, nan, nan, n...","[0.003502, nan, nan, 0.003157, nan, nan, nan, ...","[0.026, nan, nan, nan, nan, nan, nan, nan, 0.0...","[0.003092, nan, nan, nan, nan, nan, nan, nan, ..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.01492, 0.01654, 0.03436, nan, nan, 0.01553,...","[0.001747, 0.002792, 0.00246, nan, nan, 0.0031...","[0.01246, 0.01306, 0.0218, 0.0201, 0.01401, 0....","[0.000678, 0.0005302, 0.0004344, 0.000489, 0.0...","[0.01442, 0.01605, 0.02498, 0.02408, 0.0166, 0...","[0.0008626, 0.0006566, 0.000616, 0.000811, 0.0...","[0.013916, nan, nan, 0.02411, nan, nan, nan, n...","[0.003502, nan, nan, 0.003157, nan, nan, nan, ...","[0.026, nan, nan, nan, nan, nan, nan, nan, 0.0...","[0.003092, nan, nan, nan, nan, nan, nan, nan, ..."


In [27]:
LFP_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [28]:
for col in LFP_SPECTRAL_DF.columns:
    print(col)

cohort
session_dir
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
video_name
session_path
recording
current_subject
subject
all_subjects
first_timestamp
last_timestamp
video_timestamps
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace
mPFC_lfp_zscore
MD_lfp_zscore
LH_lfp_zscore
BLA_lfp_zscore
vHPC_lfp_zscore
mPFC_lfp_RMS
MD_lfp_RMS
LH_lfp_RMS
BLA_lfp_RMS
vHPC_lfp_RMS
mPFC_lfp_mask
MD_lfp_mask
LH_lfp_mask
BLA_lfp_mask
vHPC_lfp_mask
mPFC_lfp_trace_filtered
MD_lfp_trace_filtered
LH_lfp_trace_filtered
BLA_lfp_trace_filtered
vHPC_lfp_trace_filtered
mPFC_lfp_RMS_filtered
MD_lfp_RMS_filtered
LH_lfp_RMS_filtered
BLA_lfp_RMS_filtered
vHPC_lfp_RMS_filtered
mPFC_trace_power_all_frequencies_all_windows
MD_trace_power_all_frequencies_all_windows
LH_trace_power_all_frequencies_all_windows
BLA_trace_power_all_frequencies_all_windows
vHPC_trace_power_all_frequencies_all_windows
mPFC_

In [29]:
raise ValueError()

ValueError: 

- Filtering theta/gamma for coherence

In [ ]:
coherence_columns = [col for col in LFP_SPECTRAL_DF.columns if "coherence" in col and "calculation" not in col and "time" not in col]

In [ ]:
coherence_columns

In [ ]:
for col in coherence_columns:
    brain_region_name = "_".join(col.split("_")[:2])
    print(brain_region_name)
    theta_coherence_col = "{}_coherence_theta".format(brain_region_name)
    gamma_coherence_col = "{}_coherence_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_coherence_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["coherence_calculation_frequencies"] >= theta_band[0]) & (x["coherence_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_coherence_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["coherence_calculation_frequencies"] >= gamma_band[0]) & (x["coherence_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

- Filtering theta/gamma for granger

In [ ]:
granger_columns = [col for col in LFP_SPECTRAL_DF.columns if "granger" in col and "calculation" not in col and "time" not in col]

In [ ]:
granger_columns

In [ ]:
for col in granger_columns:
    brain_region_name = "-to-".join(col.split("_")[:2])
    print(brain_region_name)
    theta_granger_col = "{}_granger_theta".format(brain_region_name)
    gamma_granger_col = "{}_granger_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_granger_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["granger_calculation_frequencies"] >= theta_band[0]) & (x["granger_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_granger_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["granger_calculation_frequencies"] >= gamma_band[0]) & (x["granger_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

In [ ]:
LFP_SPECTRAL_DF.head()

In [ ]:
LFP_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))